## RAG Setup

In [ ]:
!pip install langchain pypdf huggingface_hub langchain-community langchain_ollama chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 951.2 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.1 MB/s e

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.llms import HuggingFaceHub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
import threading
import subprocess
import requests
import json

In [ ]:
!curl https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#### Run Ollama in the background

In [ ]:
def ollama() -> None:
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
!ollama pull mshojaei77/gemma3persian:latest

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling d55a281ac63d:   0% ▕▏  18 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   2% ▕▏  72 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   3% ▕▏ 103 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   4% ▕▏ 154 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   4% ▕▏ 169 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   5% ▕▏ 205 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   5% ▕▏ 221 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   6% ▕▏ 244 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   6% ▕▏ 245 MB/4.1 GB                  pulling manifest 
pulling d55a281ac63d:   6% ▕▏ 245 MB/4.1 GB

#### Load PDF with PyPDF

In [ ]:
local_path = "./Materials.pdf"
if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

data

PDF loaded successfully: ./Test1.pdf


[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-05-13T10:33:20+03:30', 'author': 'Hesam Gh', 'moddate': '2025-05-13T10:33:20+03:30', 'source': './Test1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='مواد و تغییرات آنها \nدر دنیای اطراف ما، هر چیزی که دیده می شود یا لمس می کنیم، از  ماده ساخته شده است. ماده هر  \nچیزی است که جرم دارد و فضا اشغال می کند. آب، خاک، هوا، آهن، پلاستیک، چوب، شیشه، بدن  \nانسان، مواد غذایی و... همگی نمونه هایی از ماده هستند. \nمواد از نظر ویژگی ها و حالتها با یکدیگر تفاوت دارند. سه حالت اصلی ماده عبارت اند از جامد، مایع  \nو گاز .مواد  جامد  مانند آهن، چوب یا یخ شکل مشخص دارند و به راحتی تغییر شکل نمی دهند. مواد  \nمایع  مانند آب و روغن، شکل ظرف را به خود می گیرند اما حجم ثابتی دارند. مواد گازی  مانند هوا یا  \nبخار آب، نه شکل مشخص دارند و نه حجم ثابت، و میتوانند به آسانی پخش شوند. \nتغییرات فیزیکی و شیمیایی  \nمواد در شرایط مختلف ممکن است دچار تغییر شوند. این تغییرات گاهی س

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", "  "], chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 19 chunks


In [ ]:
for i, chunk in enumerate(chunks):
  print(i, chunk.page_content)

0 مواد و تغییرات آنها 
در دنیای اطراف ما، هر چیزی که دیده می شود یا لمس می کنیم، از  ماده ساخته شده است. ماده هر  
چیزی است که جرم دارد و فضا اشغال می کند. آب، خاک، هوا، آهن، پلاستیک، چوب، شیشه، بدن
1 انسان، مواد غذایی و... همگی نمونه هایی از ماده هستند. 
مواد از نظر ویژگی ها و حالتها با یکدیگر تفاوت دارند. سه حالت اصلی ماده عبارت اند از جامد، مایع
2 و گاز .مواد  جامد  مانند آهن، چوب یا یخ شکل مشخص دارند و به راحتی تغییر شکل نمی دهند. مواد  
مایع  مانند آب و روغن، شکل ظرف را به خود می گیرند اما حجم ثابتی دارند. مواد گازی  مانند هوا یا
3 بخار آب، نه شکل مشخص دارند و نه حجم ثابت، و میتوانند به آسانی پخش شوند. 
تغییرات فیزیکی و شیمیایی  
مواد در شرایط مختلف ممکن است دچار تغییر شوند. این تغییرات گاهی ساده و قابل برگشت، و گاهی
4 عمیق و غیرقابل بازگشت هستند. این تغییرات به دو دسته ی اصلی تقسیم می شوند :تغییر فیزیکی  و   
تغییر شیمیایی. 
تغییر فیزیکی
5 تغییر شیمیایی. 
تغییر فیزیکی  
در تغییر فیزیکی، ساختمان و ماهیت ماده تغییر نمی کند، بلکه فقط شکل یا حالت آن عوض می شود.
6 برای مثال، وقتی یخ ذ

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="heydariAI/persian-embeddings")

<ipython-input-9-36bbd4105152>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="heydariAI/persian-embeddings")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

### Create VectorDB with ChromaDB

In [ ]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


In [ ]:
local_model = "mshojaei77/gemma3persian:latest"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
    شما یک مدل زبان هوشمند هستید که فقط به سوالات مربوط به دروس درسی پاسخ می‌دهد.
    وظیفه شما تولید دو نسخه متفاوت از سوال کاربر برای جستجوی دقیق‌تر اسناد مرتبط در پایگاه برداری است.
    هدف از تولید نسخه‌های متفاوت، رفع محدودیت‌های جستجوی مشابهت بر پایه فاصله است.
    لطفاً فقط در چارچوب دروس داده شده عمل کنید و از تولید سوالاتی خارج از این حوزه خودداری کنید.
    سوال اصلی: {question}
    نسخه‌های بازنویسی‌شده سوال:
    """
)
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(search_kwargs={"k": 3}),
    llm,
    prompt=QUERY_PROMPT
)

In [ ]:
template = """
تنها بر اساس متن زیر به سوال پاسخ دهید و از دانش عمومی یا اطلاعات خارج از این متن استفاده نکنید.
لطفاً فقط در حوزه دروس درسی پاسخ دهید.

متن:
{context}

سوال: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
def chat_with_bot(question : str) -> str :
    """generate answer from user question use retriever

    Args:
        question (str): user input question

    Returns:
        str: return answer and related docs 
    """
    docs = retriever.get_relevant_documents(question)

    answer = chain.invoke(question)

    source_pages = "\n".join([doc.page_content.strip() for doc in docs])

    return answer, source_pages


In [ ]:
chat_with_bot("تفاوت بین تغییر فیزیکی و تغییر شیمیایی را با مثال توضیح بده")


 Sources:

 Source 1:
تغییر شیمیایی. 
تغییر فیزیکی  
در تغییر فیزیکی، ساختمان و ماهیت ماده تغییر نمی کند، بلکه فقط شکل یا حالت آن عوض می شود.
----------------------------------------

 Source 2:
عمیق و غیرقابل بازگشت هستند. این تغییرات به دو دسته ی اصلی تقسیم می شوند :تغییر فیزیکی  و   
تغییر شیمیایی. 
تغییر فیزیکی
----------------------------------------

 Source 3:
آب دوباره می تواند به مایع تبدیل شود). 
تغییر شیمیایی  
در تغییر شیمیایی، مادهی جدیدی با خواص متفاوت به وجود می آید. در این نوع تغییرات، ساختار
----------------------------------------
در تغییر شیمیایی، مادهی جدیدی با خواص متفاوت به وجود می آید. در این نوع تغییرات، ساختار. در مقابل، در تغییر فیزیکی، ساختمان و ماهیت ماده تغییر نمی کند، بلکه فقط شکل یا حالت آن عوض می شود. به عنوان مثال، آب دوباره می تواند به مایع تبدیل شود.


## Backend setup

In [ ]:
!pip install flask-ngrok flask-cors

In [ ]:
!ngrok authtoken YOUR_API_KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request

app = Flask(__name__)

from pyngrok import ngrok

public_url = ngrok.connect(5000)
print('Public URL:', public_url)

run_with_ngrok(app)

Public URL: NgrokTunnel: "https://4a00-34-125-186-65.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify, Response
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

@app.route("/ask", methods=["POST"])
def chat() -> Response:
    data = request.get_json()
    question = data.get("question")

    if not data:
        return jsonify({"error": "No json data!!"}), 400

    question = data.get("question")
    if not question:
        return jsonify({"error": "Question is required!!"}), 400

    try:
        answer, source_pages = chat_with_bot(question)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    response = {
        "question": question,
        "answer": answer,
        "source_pages": source_pages
    }

    return jsonify(response)

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4a00-34-125-186-65.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 16:37:33] "OPTIONS /ask HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 16:37:42] "POST /ask HTTP/1.1" 200 -
